# 双向循环神经网络
在序列学习中，我们总是根据过去的序列$(\mathbf{X}_1, \ldots , \mathbf{X}_{t-1})$去预测$\mathbf{X}_{t}$，但是这并不是唯一的情况，在完形填空中，我们需要对上下文进行检索，从而完成结合上下文的词语预测。



# 隐马尔科夫模型中的DP

如果想用概率图模型来解决序列预测的问题，可以设计一个隐变量模型：在任意时间步$t$，假设存在某个隐变量$h_t$，通过概率$P(x_t \mid h_t)$来控制我们观测到的$x_t$，此外，任何$h_t \to h_{t+1}$转移都是有一些状态转移概率$P(h_{t+1} \mid h_t)$ 给出。这个概率图模型就是一个隐马尔可夫模型。

因此，对于有$T$个观测值的序列，我们在观测状态和隐状态上具有以下联合概率分布：
$$P(x_1,\ldots,x_T,h_1,\ldots,h_T) = \prod_{t=1}^T P(h_t \mid h_{t-1})P(x_t \mid h_t), \text{where} P(h_1|h_0)=P(h_1)$$

假设我们观测到了所有的$x_i$除了$x_j$，并且我们的目标是计算$P(x_j \mid x_{-j})$，其中$x_{-j}=(x_1, \ldots, x_{j-1},x_{j+1},\ldots,x_T)$，由于$P(x_j \mid x_{-j})$没有隐变量，因此我们考虑对$h_1,\ldots,h_T$选择构成的所有可能的组合进行求和。

如果任何个$h_i$可以接受$k$个不同的值，我们需要对$k^T$个项进行求和，但是我们可以使用DP来解决这个问题：
$$
\begin{split}\begin{aligned}
    &P(x_1,\ldots,x_T)\\
    =& \sum_{h_1,\ldots,h_T}P(x_1,\ldots,x_T,h_1,\ldots,h_T)\\
    =& \sum_{h_1,\ldots,h_T} \prod_{t=1}^{T}P(h_t \mid h_{t-1})P(x_t \mid h_t)\\
    =& \sum_{h_2,\ldots,h_T} \underbrace {\left[\sum_{h_1} P(h_1)P(x_1\mid h_1)P(h_2\mid h_1)\right]}_{\pi_2(h_2) \stackrel{\mathrm{def}}{=}} P(x_2\mid h_2) \prod_{t=3}^{T} P(h_t \mid h_{t-1})P(x_t\mid h_t)\\
    =& \sum_{h_3,\ldots,h_T} \underbrace {\left[\sum_{h_2} \pi_2(h_2) P(x_2\mid h_2)P(h_3\mid h_2)\right]}_{\pi_3(h_3) \stackrel{\mathrm{def}}{=}} P(x_3\mid h_3) \prod_{t=4}^{T} P(h_t \mid h_{t-1})P(x_t\mid h_t)\\
    =& \dots\\
    =& \sum_{hT} \pi_T(h_T)P(x_T \mid h_T)
\end{aligned}\end{split}
$$

通常，我们将“前项递归” forward recursion 写为：
$$\pi_{t+1}(h_{t+1}) = \sum_{ht} \pi_t(h_t)P(x_t\mid h_t)P(h_{t+1}\mid h_t).$$
其中 $ \pi_1(h_1) = P(h_1)$，上式还能够简写成$\pi_{t+1}=f(\pi_t,x_t)$,这就向我们在RNN中提到的隐变量更新方程。

对于后向递归也有一样的:
$$\rho_{t-1}(h_{t-1})= \sum_{ht} P(h_t \mid h_{t-1}) P(x_t \mid h_t) \rho_t(h_t)$$

`双向循环神经网络`是添加了反向传递信息的隐藏层的网络，它于1997年被提出，输出需要对两个方向的隐状态进行拼接(concatenate)得到输出层的隐状态 $\mathbf{H}_t \in \mathbb{R}^{n \times 2h}$

双向RNNs只适用于某些场合，计算速度很慢，因为在网络前向传播需要在双向层中进行前向和后向递归，并且网络的反向传播还依赖于前向传播的结果，因此梯度求解很复杂。通常运用于填充单词，词元注释或者机器翻译等等，在NLP中会使用BiRNNs来编码文本序列。

由于双向循环网络使用了过去和未来的数据，即使它被训练的很好，我们也不能使用它来进行预测任务，因为我们没有后面的序列。

# 使用双向LSTM

In [ ]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F

lstm_layer  =nn.LSTM(35,512,2,bidirectional=True)
# 要使用双向的网络，我们只需要在模型参数里面加上bidirectional=True
